### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.13.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/Btech/dataSet.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataSet/trainingData/P/315.jpg  
  inflating: dataSet/trainingData/P/316.jpg  
  inflating: dataSet/trainingData/P/317.jpg  
  inflating: dataSet/trainingData/P/318.jpg  
  inflating: dataSet/trainingData/P/32.jpg  
  inflating: dataSet/trainingData/P/320.jpg  
  inflating: dataSet/trainingData/P/321.jpg  
  inflating: dataSet/trainingData/P/322.jpg  
  inflating: dataSet/trainingData/P/323.jpg  
  inflating: dataSet/trainingData/P/324.jpg  
  inflating: dataSet/trainingData/P/325.jpg  
  inflating: dataSet/trainingData/P/327.jpg  
  inflating: dataSet/trainingData/P/328.jpg  
  inflating: dataSet/trainingData/P/329.jpg  
  inflating: dataSet/trainingData/P/33.jpg  
  inflating: dataSet/trainingData/P/330.jpg  
  inflating: dataSet/trainingData/P/331.jpg  
  inflating: dataSet/trainingData/P/332.jpg  
  inflating: dataSet/trainingData/P/333.jpg  
  inflating: dataSet/trainingData/P/335.jpg  
  inflating: dataSet/trainingDa

Function to normalize image according to pre-trained mode.

In [5]:
def normalize_data(img):


    #Normalize for ResNet50
  return tf.keras.applications.resnet50.preprocess_input(img)

In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function= normalize_data,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [7]:
test_datagen = ImageDataGenerator(preprocessing_function= normalize_data)

### Creating the Training set

In [10]:
training_set = train_datagen.flow_from_directory('/content/dataSet/trainingData',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [11]:
test_set = test_datagen.flow_from_directory('/content/dataSet/testingData',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

In [35]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(128,128,3),
                                       weights='imagenet')

In [36]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [37]:
len(model.layers)

175

In [38]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [39]:
#Set last few layers to trainable
for layer in model.layers[173:]:
    layer.trainable = True

In [40]:
#get Output layer of Pre0trained model
x1 = model.output

#Global average pool to reduce number of features and Flatten the output
x2 = tf.keras.layers.GlobalAveragePooling2D()(x1)

In [41]:
# adding extra layers
x3 = tf.keras.layers.Dense(1024,activation='relu',kernel_initializer= "he_uniform")(x2)
x4 = tf.keras.layers.Dropout(0.4)(x3)
x5=  tf.keras.layers.Dense(512,activation='relu',kernel_initializer= "he_uniform")(x4)


In [42]:
#Add output layer
prediction = tf.keras.layers.Dense(27,activation='softmax')(x5)

## Building final Model for Classification

In [43]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [44]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
#How does our overall /model looks
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [46]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [47]:
final_model.fit(training_set,
                  epochs = 30, steps_per_epoch=12845//32,
                validation_steps=4268//32,
                  validation_data = test_set,callbacks = [model_checkpoint_callback])

Epoch 1/30
401/401 [==============================] - ETA: 0s - loss: 0.4558 - accuracy: 0.8619
Epoch 1: val_accuracy improved from -inf to 0.90273, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


401/401 [==============================] - 102s 220ms/step - loss: 0.4558 - accuracy: 0.8619 - val_loss: 0.2833 - val_accuracy: 0.9027
Epoch 2/30
401/401 [==============================] - ETA: 0s - loss: 0.1548 - accuracy: 0.9497
Epoch 2: val_accuracy improved from 0.90273 to 0.97110, saving model to model.h5
401/401 [==============================] - 85s 212ms/step - loss: 0.1548 - accuracy: 0.9497 - val_loss: 0.0825 - val_accuracy: 0.9711
Epoch 3/30
401/401 [==============================] - ETA: 0s - loss: 0.1197 - accuracy: 0.9618
Epoch 3: val_accuracy improved from 0.97110 to 0.98684, saving model to model.h5
401/401 [==============================] - 85s 212ms/step - loss: 0.1197 - accuracy: 0.9618 - val_loss: 0.0414 - val_accuracy: 0.9868
Epoch 4/30
401/401 [==============================] - ETA: 0s - loss: 0.0990 - accuracy: 0.9685
Epoch 4: val_accuracy did not improve from 0.98684
401/401 [==============================] - 84s 208ms/step - loss: 0.0990 - accuracy: 0.9685 - va

In [49]:
model_json = final_model.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
final_model.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [50]:
from google.colab import files
files.download('model_new.json')
files.download('model_new.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# END OF TRANSFER LEARNING

# Simple Custom Model

#### Initializing the CNN

In [ ]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3,
                                     padding="same",
                                     activation="relu",
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [ ]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

#### Adding a second convolutional layer

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                      kernel_size=3,
                                      padding="same",
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

#### Step 3 - Flattening

In [ ]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [ ]:
classifier.add(tf.keras.layers.Dense(units=128,
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [ ]:
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
classifier.summary()

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
classifier.fit(training_set,
                  epochs = 30, batch_size = 32,
                  validation_data = test_set,callbacks = [model_checkpoint_callback])

#### Saving the Model

In [ ]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')